# PCA-based Heart Model to Image Registration Experiment

This notebook demonstrates using the `RegisterModelToImagePCA` class to register
a statistical shape model to patient CT images using PCA-based shape variation.

## Overview
- Uses the KCL Heart Model PCA statistical shape model
- Registers to the same Duke Heart CT data as the original notebook
- Two-stage optimization: rigid alignment + PCA shape fitting
- Converts segmentation mask to intensity image for registration

## Setup and Imports

In [ ]:
# PCA-based Heart Model to Image Registration Experiment

import json
import os
from pathlib import Path

import itk
import numpy as np
import pyvista as pv
from itk import TubeTK as ttk

# Import from PhysioMotion4D package
from physiomotion4d import (
    ContourTools,
    RegisterModelToImagePCA,
    RegisterModelToModelICP,
    TransformTools,
)


## Define File Paths

In [ ]:
# Patient CT image (defines coordinate frame)
patient_data_dir = Path.cwd().parent.parent / 'data' / 'Slicer-Heart-CT'
patient_ct_path = patient_data_dir / 'slice_007.mha'
patient_ct_heart_mask_path = patient_data_dir / 'slice_007_heart_mask4.nii.gz'

# PCA heart model data
model_data_dir = Path.cwd().parent.parent / 'data' / 'KCL-Heart-Model' / 'pca'
average_model_path = model_data_dir / 'pca_All_mean.vtk'
pca_json_path = model_data_dir / 'pca.json'
pca_group_key = 'All'

# Output directory
output_dir = Path.cwd() / 'results_pca'
os.makedirs(output_dir, exist_ok=True)

print(f"Patient data: {patient_data_dir}")
print(f"Model data: {model_data_dir}")
print(f"Output directory: {output_dir}")

## Load and Preprocess Patient Image

In [ ]:
# Load patient CT image
print("Loading patient CT image...")
fixed_image = itk.imread(str(patient_ct_path))
print(f"  Original size: {itk.size(fixed_image)}")
print(f"  Original spacing: {itk.spacing(fixed_image)}")

# Resample to 1mm isotropic spacing
print("Resampling to 1mm isotropic...")
resampler = ttk.ResampleImage.New(Input=fixed_image)
resampler.SetSpacing([1.0, 1.0, 1.0])
resampler.Update()
fixed_image = resampler.GetOutput()

print(f"  Resampled size: {itk.size(fixed_image)}")
print(f"  Resampled spacing: {itk.spacing(fixed_image)}")

# Save preprocessed image
itk.imwrite(fixed_image, str(output_dir / 'fixed_image.mha'), compression=True)
print(f"✓ Saved preprocessed image")

## Load and Process Heart Segmentation Mask

In [ ]:
# Load heart segmentation mask
print("Loading heart segmentation mask...")
heart_mask_image = itk.imread(str(patient_ct_heart_mask_path))
print(f"  Mask size: {itk.size(heart_mask_image)}")
print(f"  Mask spacing: {itk.spacing(heart_mask_image)}")

In [ ]:
# Handle image orientation (flip if needed)
flip0 = np.array(heart_mask_image.GetDirection())[0,0] < 0
flip1 = np.array(heart_mask_image.GetDirection())[1,1] < 0
flip2 = np.array(heart_mask_image.GetDirection())[2,2] < 0

if flip0 or flip1 or flip2:
    print(f"Flipping image axes: {flip0}, {flip1}, {flip2}")

    # Flip CT image
    flip_filter = itk.FlipImageFilter.New(Input=fixed_image)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    fixed_image = flip_filter.GetOutput()
    id_mat = itk.Matrix[itk.D, 3, 3]()
    id_mat.SetIdentity()
    fixed_image.SetDirection(id_mat)

    # Flip mask image
    flip_filter = itk.FlipImageFilter.New(Input=heart_mask_image)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    heart_mask_image = flip_filter.GetOutput()
    heart_mask_image.SetDirection(id_mat)

    print("✓ Images flipped to standard orientation")

# Save oriented images
itk.imwrite(fixed_image, str(output_dir / 'fixed_image_oriented.mha'), compression=True)
itk.imwrite(heart_mask_image, str(output_dir / 'heart_mask_oriented.mha'), compression=True)

## Convert Segmentation Mask to an Edge-Distance Intensity Image

For PCA registration, we need an intensity-based target. We'll convert the binary
segmentation mask to a smooth intensity image using distance transforms and Gaussian blurring.

In [ ]:
print("Converting segmentation mask to intensity image...")

# Convert mask to binary
mask_arr = itk.GetArrayFromImage(heart_mask_image)
binary_mask_arr = (mask_arr > 0).astype(np.uint8)
binary_mask_image = itk.GetImageFromArray(binary_mask_arr)
binary_mask_image.CopyInformation(heart_mask_image)

edge_filter = itk.BinaryContourImageFilter.New(Input=binary_mask_image)
edge_filter.SetForegroundValue(1)
edge_filter.SetBackgroundValue(0)
edge_filter.SetFullyConnected(False)
edge_filter.Update()
edge_mask_image = edge_filter.GetOutput()

# Compute signed distance map (positive inside, negative outside)
print("  Computing signed distance map...")
distance_filter = itk.SignedMaurerDistanceMapImageFilter.New(Input=edge_mask_image)
distance_filter.SetSquaredDistance(False)
distance_filter.SetUseImageSpacing(True)
distance_filter.SetInsideIsPositive(False)
distance_filter.Update()
distance_image = distance_filter.GetOutput()

# Normalize to [0, 1000] range for better optimization
print("  Normalizing intensity values...")
dist_arr = itk.GetArrayFromImage(distance_image)
min_val = dist_arr.min()
max_val = dist_arr.max()
normalized_arr = ((1.0 - (dist_arr - min_val) / (max_val - min_val)) * 100.0).astype(np.float32)
target_image = itk.GetImageFromArray(normalized_arr)
target_image.CopyInformation(distance_image)

# Save intermediate and final images
itk.imwrite(binary_mask_image, str(output_dir / 'binary_mask.mha'), compression=True)
itk.imwrite(distance_image, str(output_dir / 'distance_map.mha'), compression=True)
itk.imwrite(target_image, str(output_dir / 'target_intensity_image.mha'), compression=True)

print(f"✓ Target intensity image created")
print(f"  Min intensity: {normalized_arr.min():.2f}")
print(f"  Max intensity: {normalized_arr.max():.2f}")
print(f"  Mean intensity: {normalized_arr.mean():.2f}")

## Load PCA Heart Model

In [ ]:
# Load the average model
print("Loading PCA heart model...")
average_mesh_original = pv.read(str(average_model_path))
print(f"  Average model: {average_mesh_original.n_points} points, {average_mesh_original.n_cells} cells")
print(f"  Model bounds: {average_mesh_original.bounds}")
print(f"  Model center: {average_mesh_original.center}")

# Save a copy for reference
average_mesh_original.save(str(output_dir / 'average_model_original.vtk'))

# Load PCA data from JSON
print(f"\nLoading PCA data from JSON...")
with open(str(pca_json_path), 'r') as f:
    pca_data = json.load(f)

# Extract PCA group data
group_data = pca_data[pca_group_key]

# Extract eigenvalues and convert to standard deviations
eigenvalues = np.array(group_data['eigenvalues'])
std_deviations = np.sqrt(eigenvalues)
print(f"  Loaded {len(std_deviations)} eigenvalues (converted to std deviations)")

# Extract eigenvector components
eigenvectors = np.array(group_data['components'], dtype=np.float64)
print(f"  Loaded eigenvectors with shape {eigenvectors.shape}")
print(f"  ✓ PCA data loaded successfully!")

## Perform Initial ICP Affine Registration

Use ICP (Iterative Closest Point) with affine mode to align the model surface to the patient surface extracted from the segmentation mask. This provides a good initial alignment for the PCA-based registration.

The ICP registration pipeline:
1. Centroid alignment (automatic)
2. Rigid ICP alignment
3. Affine ICP alignment (allows scaling and shearing)

The PCA registration will then refine this initial alignment with shape model constraints.

In [ ]:
# Perform initial ICP-based affine registration to patient surface
print("Performing initial ICP affine registration...")
print("="*70)

# Extract surface from patient mask for ICP target

contour_tools = ContourTools()
patient_surface = contour_tools.extract_contours(heart_mask_image)
print(f"  Extracted patient surface: {patient_surface.n_points} points")

# Extract surface from average model for ICP source
model_surface = average_mesh_original.extract_surface()
print(f"  Extracted model surface: {model_surface.n_points} points")

# Perform ICP affine registration
icp_registrar = RegisterModelToModelICP(
    moving_mesh=model_surface,
    fixed_mesh=patient_surface
)

icp_result = icp_registrar.register(mode='rigid', max_iterations=200)

# Get the aligned mesh and transform
aligned_model_surface = icp_result['moving_mesh']
phi_FM = icp_result['phi_FM']

print("\n✓ ICP affine registration complete")
print("  Initial alignment obtained using centroid + rigid + ICP")

# Save aligned model for visualization
aligned_model_surface.save(str(output_dir / 'icp_aligned_model_surface.vtp'))
print("  Saved ICP-aligned model surface")

# Apply ICP transform to the full average mesh (not just surface)
# This gets the volumetric mesh into patient space for PCA registration
transform_tools = TransformTools()
average_mesh_icp_aligned = transform_tools.transform_pvcontour(
    average_mesh_original,
    phi_FM
)
print("\n✓ Applied ICP transform to full average mesh")
print(f"  Aligned mesh center: {average_mesh_icp_aligned.center}")

# Save ICP-aligned full mesh
average_mesh_icp_aligned.save(str(output_dir / 'average_model_icp_aligned.vtk'))

## Initialize PCA Registration

In [ ]:
print("Initializing RegisterModelToImagePCA...")
print("="*70)

# Use the ICP-aligned mesh as the starting point for PCA registration
pca_registrar = RegisterModelToImagePCA(
    average_mesh=average_mesh_icp_aligned,
    eigenvectors=eigenvectors,
    std_deviations=std_deviations,
    reference_image=target_image
)

print("✓ PCA registrar initialized")
print("  Using ICP-aligned mesh as starting point")
print(f"  Number of points: {len(pca_registrar.average_mesh.points)}")
print(f"  Number of PCA modes: {pca_registrar.n_pca_modes}")
print(f"  Reference image size: {itk.size(target_image)}")

## Run PCA-Based Shape Optimization

Now that we have a good initial alignment from ICP affine registration, we run the PCA-based registration to optimize the shape parameters while allowing small rigid refinements.

The PCA registration pipeline includes:
1. **Stage 1**: Minor rigid refinement (starting from ICP-aligned position)
2. **Stage 2**: Joint optimization of rigid parameters + PCA shape coefficients

Since the ICP already provided good alignment, we use reduced bounds for rigid refinement and focus on PCA shape optimization.


In [ ]:
print("\n" + "="*70)
print("PCA-BASED SHAPE OPTIMIZATION")
print("="*70)
print("\nRunning complete PCA registration pipeline...")
print("  (Starting from ICP-aligned mesh, so skipping rigid stage)")
print("  Using identity transform as initial guess")

# Run complete registration
# Since we already have good alignment from ICP, we can focus on PCA shape fitting
result = pca_registrar.register(
    n_pca_modes=10,  # Use first 10 PCA modes
    stage1_max_iterations=10,  # Fewer iterations for rigid since already aligned
    stage2_max_iterations=200,  # More iterations for PCA optimization
    pca_coefficient_bounds=3.0,  # ±3 std deviations per mode
    rigid_refinement_bounds={'versor': 0.1, 'translation_mm': 10.0}  # Small refinements only
)

print("\n✓ PCA registration complete")


### Display Registration Results

Review the optimization results from the PCA registration pipeline.


In [ ]:
print("\n" + "="*70)
print("REGISTRATION RESULTS")
print("="*70)

# Display results
print(f"\nFinal Registration Metrics:")
print(f"  Final mean intensity: {result['intensity']:.4f}")

print(f"\nOptimized Rigid Coefficients:")
print(result['pre_phi_FM'])
print(f"\nOptimized PCA Coefficients (in units of std deviations):")
for i, coef in enumerate(result['pca_coefficients_FM']):
    print(f"  Mode {i+1:2d}: {coef:7.4f}")

# Get the final registered mesh
registered_mesh = result['registered_mesh']
print(f"\nRegistered Mesh Properties:")
print(f"  Number of points: {registered_mesh.n_points}")
print(f"  Number of cells: {registered_mesh.n_cells}")
print(f"  Center: {registered_mesh.center}")
print(f"  Bounds: {registered_mesh.bounds}")

print("\n✓ Registration pipeline complete!")


## Save Registration Results


In [ ]:
print("\nSaving results...")

# Save final PCA-registered mesh
registered_mesh.save(str(output_dir / 'registered_mesh_pca.vtk'))
print(f"  Saved final PCA-registered mesh")

# Save ICP-aligned mesh for comparison
average_mesh_icp_aligned.save(str(output_dir / 'registered_mesh_icp_only.vtk'))
print(f"  Saved ICP-only aligned mesh")

# Save patient surface
patient_surface.save(str(output_dir / 'patient_surface.vtp'))
print(f"  Saved patient surface")

# Save transforms
itk.transformwrite([phi_FM], str(output_dir / 'icp_rigid_transform.hdf'), compression=True)
itk.transformwrite([result['pre_phi_FM']], str(output_dir / 'pca_pre_rigid_transform.hdf'), compression=True)
print(f"  Saved transforms")

# Save PCA coefficients
np.savetxt(
    str(output_dir / 'pca_coefficients.txt'),
    result['pca_coefficients_FM'],
    header=f"PCA coefficients for {len(result['pca_coefficients_FM'])} modes"
)
print(f"  Saved PCA coefficients")

print(f"\n✓ All results saved to: {output_dir}")


## Visualize Results


In [ ]:
# Extract surface from patient mask
patient_surface = ContourTools().extract_contours(heart_mask_image)
patient_surface.save(str(output_dir / 'patient_surface.vtp'))

# Create side-by-side comparison
plotter = pv.Plotter(shape=(1, 2), window_size=[1000, 600])

plotter.subplot(0, 0)
plotter.add_mesh(patient_surface, color='red', opacity=1.0, label='Patient')
plotter.add_mesh(average_mesh_icp_aligned, color='green', opacity=0.6, label='ICP Registered')
plotter.add_title('After PCA Shape Fitting')
plotter.add_axes()

# After PCA shape fitting
plotter.subplot(0, 1)
plotter.add_mesh(patient_surface, color='red', opacity=1.0, label='Patient')
plotter.add_mesh(registered_mesh, color='green', opacity=0.6, label='PCA Registered')
plotter.add_title('After PCA Shape Fitting')
plotter.add_axes()

plotter.link_views()
plotter.show()
